In [ ]:
import polars as pl
from sklearn.model_selection import train_test_split

from data_shaping import build_dataset
from model import build_feature_df, import_pipeline

oecd_csv = ('data/final/oecd.csv')
acled_csv = ('data/final/acled.csv')
data_csv = ('data/final/data.csv')
model_pkl = ('random_forest.pkl')
data = build_dataset(oecd_csv, acled_csv, data_csv, read_data_csv=False)
X = data.drop(['iso', 'year_month', 'incidents'])
y = data['incidents']
X_train, X_test, y_train, y_test = train_test_split(X, y)
pipe = import_pipeline(X_train, y_train, model_pkl)
coeffs = build_feature_df(pipe)
if isinstance(coeffs, pl.DataFrame):
    coeffs.plot.bar(x='feature', y='importance')

alt.Chart(...)

### Global Acute Food Insecurity
* https://data.humdata.org/dataset/global-acute-food-insecurity-country-data
* Only covers developing nations, not compatible with current OECD countries

In [ ]:
#ipc = pl.read_csv('data/ipc_global_national_long_latest_food_insecurity.csv')

### Global Mobility Report (Google):
* https://data.humdata.org/dataset/google-mobility-report
* Only covers 2020-2022

In [218]:
#global_mobility_report = pl.read_csv('data/inspected/Global_Mobility_Report.csv')

In [219]:
nightsky = pl.read_csv('data/ntl_adm2_monthly_2016.csv', null_values='NA')

In [7]:
nightsky

ISO_A3,ADM1CD_c,ADM2CD_c,NAM_0,NAM_1,NAM_2,date,n_gasflaring_locs,ntl_quality_prop_na,ntl_quality_prop_0_good,ntl_quality_prop_1_poor,ntl_quality_prop_2_gapfilled,ntl_sum,ntl_mean,ntl_median,ntl_max,ntl_q05,ntl_q95,ntl_gf_5km_sum,ntl_gf_5km_mean,ntl_gf_5km_median,ntl_gf_5km_max,ntl_gf_5km_q05,ntl_gf_5km_q95,ntl_nogf_5km_sum,ntl_nogf_5km_mean,ntl_nogf_5km_median,ntl_nogf_5km_max,ntl_nogf_5km_q05,ntl_nogf_5km_q95,ntl_gf_10km_sum,ntl_gf_10km_mean,ntl_gf_10km_median,ntl_gf_10km_max,ntl_gf_10km_q05,ntl_gf_10km_q95,ntl_nogf_10km_sum,ntl_nogf_10km_mean,ntl_nogf_10km_median,ntl_nogf_10km_max,ntl_nogf_10km_q05,ntl_nogf_10km_q95
str,str,str,str,str,str,str,i64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,str,str,str,str,str,str,f64,f64,f64,f64,f64,f64,str,str,str,str,str,str,f64,f64,f64,f64,f64,f64
"""ABW""","""ABW001""","""ABW001001""","""Aruba (Neth.)""","""Administrative unit not availa…","""Administrative unit not availa…","""2016-01-01""",0,0.0,0.874351,0.125649,0.0,5212.629395,6.044495,2.521918,90.729057,0.070298,22.623181,null,null,null,null,null,null,5212.629395,6.044495,2.521918,90.729057,0.070298,22.623181,null,null,null,null,null,null,5212.629395,6.044495,2.521918,90.729057,0.070298,22.623181
"""ABW""","""ABW001""","""ABW001001""","""Aruba (Neth.)""","""Administrative unit not availa…","""Administrative unit not availa…","""2016-02-01""",0,0.0,0.906542,0.09242,0.001038,5044.547852,5.84959,2.771983,84.466125,0.096552,20.592231,null,null,null,null,null,null,5044.547852,5.84959,2.771983,84.466125,0.096552,20.592231,null,null,null,null,null,null,5044.547852,5.84959,2.771983,84.466125,0.096552,20.592231
"""ABW""","""ABW001""","""ABW001001""","""Aruba (Neth.)""","""Administrative unit not availa…","""Administrative unit not availa…","""2016-03-01""",0,0.0,0.877466,0.122534,0.0,5555.727051,6.442346,3.145681,124.96019,0.181422,21.818279,null,null,null,null,null,null,5555.727051,6.442346,3.145681,124.96019,0.181422,21.818279,null,null,null,null,null,null,5555.727051,6.442346,3.145681,124.96019,0.181422,21.818279
"""ABW""","""ABW001""","""ABW001001""","""Aruba (Neth.)""","""Administrative unit not availa…","""Administrative unit not availa…","""2016-04-01""",0,0.0,0.0,1.0,0.0,5187.452148,6.0153,2.89655,69.404099,0.24911,20.452081,null,null,null,null,null,null,5187.452148,6.0153,2.89655,69.404099,0.24911,20.452081,null,null,null,null,null,null,5187.452148,6.0153,2.89655,69.404099,0.24911,20.452081
"""ABW""","""ABW001""","""ABW001001""","""Aruba (Neth.)""","""Administrative unit not availa…","""Administrative unit not availa…","""2016-05-01""",0,0.0,0.0,0.876428,0.123572,5010.268555,5.80984,3.408621,94.178596,0.210237,19.1727,null,null,null,null,null,null,5010.268555,5.80984,3.408621,94.178596,0.210237,19.1727,null,null,null,null,null,null,5010.268555,5.80984,3.408621,94.178596,0.210237,19.1727
…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…
"""ZWE""","""ZWE008""","""ZWE008003""","""Zimbabwe""","""Matabeleland North""","""Hwange""","""2016-12-01""",0,0.000047,0.0,0.683857,0.316095,1830.144897,0.012482,0.251133,105.533417,0.025113,0.477152,null,null,null,null,null,null,1830.144897,0.012482,0.251133,105.533417,0.025113,0.477152,null,null,null,null,null,null,1830.144897,0.012482,0.251133,105.533417,0.025113,0.477152
"""ZWE""","""ZWE009""","""ZWE009005""","""Zimbabwe""","""Matabeleland South""","""Matobo""","""2016-12-01""",0,0.0,0.000108,0.96191,0.037982,153.748627,0.004247,0.252224,28.51372,0.025222,0.479225,null,null,null,null,null,null,153.748627,0.004247,0.252224,28.51372,0.025222,0.479225,null,null,null,null,null,null,153.748627,0.004247,0.252224,28.51372,0.025222,0.479225
"""ZWE""","""ZWE009""","""ZWE009007""","""Zimbabwe""","""Matabeleland South""","""Bulilima""","""2016-12-01""",0,0.0,0.000526,0.987076,0.012397,85.868095,0.002562,0.256001,3.067989,0.0256,0.486401,null,null,null,null,null,null,85.868095,0.002562,0.256001,3.067989,0.0256,0.486401,null,null,null,null,null,null,85.868095,0.002562,0.256001,

In [4]:
nightsky_date = year_month_col(nightsky, 'date')

In [6]:
nightsky_date['ISO_A3', 'ntl_sum', 'year_month'].group_by('ISO_A3', 'year_month').mean()

ISO_A3,year_month,ntl_sum
str,str,f64
"""BRB""","""2016-05""",927.857848
"""BRN""","""2016-09""",1757.215505
"""BTN""","""2016-10""",37.516818
"""BVT""","""2016-07""",1.92469
"""CCK""","""2016-03""",0.0
…,…,…
"""ZAF""","""2016-01""",29690.783609
"""ZAF""","""2016-05""",30393.661642
"""ZAF""","""2016-12""",30225.746505
